这个文件主要解决两个事情：

1. 自定义 autograd function
2. 自定义 nn modules 

# Custom autograd function

$P_3(x) = \frac{1}{2} (5 x^3 - 3 x)$

求导为
$P\prime_3(x) = \frac{3}{2}(5x^2 - 1)$

1. 继承torch.autograd.Function
2. 构建forward和backward函数
    2.1 forward函数接收ctx和输入张量input，返回输出张量
    2.2 backward函数接收ctx和梯度张量grad_output，返回输入张量的梯度张量


如何理解grad_output?

当你在神经网络中执行反向传播时，`backward()`函数会根据损失函数相对于网络参数的梯度来更新这些参数。

`backward()`函数通常接受两个参数：

1. `ctx`：这是一个上下文对象，它通常包含了执行反向传播所需的所有信息。在某些自动微分系统中，这个上下文对象可能包含了前向传播中的中间计算结果，以便在反向传播时使用。

2. `grad_output`：这是输出梯度，即损失函数相对于网络最后一层输出的梯度。这个梯度指示了输出误差的大小和方向，它是通过网络的反向传播计算得到的。

grad_output的数学表示

在深度学习中，`grad_output`通常表示为损失函数关于网络输出的梯度。如果我们用数学公式来表示这个概念，它看起来会是这样的：

假设我们有一个损失函数 $L$，它是网络输出 \( \hat{y} \)（即 \( \text{predictions} \)）和真实目标值 \( y \) 的函数，可以写作：

\[ L(\hat{y}, y) \]

`grad_output` 是这个损失函数 \( L \) 对网络输出 \( \hat{y} \) 的导数。在数学符号中，我们可以将其表示为：

\[ \frac{\partial L}{\partial \hat{y}} \]

这个导数（梯度）告诉我们，为了减少损失 \( L \)，我们需要在哪个方向上调整网络的输出 \( \hat{y} \)。在实际的神经网络实现中，这个梯度是通过自动微分系统自动计算的，比如 PyTorch 中的 `.backward()` 方法。

举个例子，如果我们使用的是均方误差损失（Mean Squared Error, MSE），它的形式是：

\[ L(\hat{y}, y) = \frac{1}{2} \sum (y_i - \hat{y}_i)^2 \]

其中 \( y_i \) 是真实目标值，\( \hat{y}_i \) 是预测值，求和是对所有样本 \( i \) 进行的。对于单个样本的 MSE 损失，我们可以写成：

\[ L(\hat{y}, y) = \frac{1}{2} (y - \hat{y})^2 \]

对于这个 MSE 损失函数，`grad_output` 就是 \( L \) 对 \( \hat{y} \) 的导数，计算如下：

\[ \frac{\partial L}{\partial \hat{y}} = y - \hat{y} \]

这个导数告诉我们，为了减少损失，我们需要将预测值 \( \hat{y} \) 调整得更接近真实目标值 \( y \)。在反向传播过程中，这个梯度会被用来更新网络中的权重和偏置，以便在下一次前向传播时产生更准确的预测。



让我们通过一个简单的例子来说明`grad_output`的概念：

假设你有一个简单的神经网络，它由一个输入层、一个隐藏层和一个输出层组成。网络的目的是预测一个二元分类问题的结果。


```python
import torch

# 假设输入数据和目标标签
inputs = torch.tensor([[1.0, 2.0], [3.0, 4.0]])
targets = torch.tensor([1, 0])

# 神经网络的参数（这里简化为只有两个权重和一个偏置）
weights = torch.randn(2, 1, requires_grad=True)
bias = torch.randn(1, requires_grad=True)

# 前向传播
def forward(inputs, weights, bias):
    outputs = torch.mm(inputs, weights) + bias  # 在PyTorch中，torch.mm是一个矩阵乘法函数，用于执行两个二维矩阵的乘法。mm代表矩阵乘法（Matrix Multiplication）。
    return outputs

# 计算预测值
predictions = forward(inputs, weights, bias)

# 计算损失（这里使用二元交叉熵损失）
loss_function = torch.nn.BCELoss()
loss = loss_function(predictions, targets)

# 反向传播
loss.backward()

# 打印梯度
print(weights.grad)  # 这将显示权重的梯度
print(bias.grad)    # 这将显示偏置的梯度
```

在这个例子中，`loss.backward()`调用会自动计算`loss`相对于`weights`和`bias`的梯度。`grad_output`在这个上下文中就是`loss`相对于`predictions`（即网络输出）的梯度。因为`loss`是通过对`predictions`应用损失函数得到的，所以`grad_output`实际上是损失函数输出的导数，它告诉我们如何调整网络的权重和偏置来减少损失。

当我们调用`loss.backward()`时，我们实际上是在告诉PyTorch：“我有一个损失值，我需要计算它相对于所有可训练参数的梯度。”PyTorch会自动计算这些梯度，并将它们存储在`.grad`属性中，以便我们可以使用它们来更新网络的参数。





In [ ]:
import torch
import math

class LegendrePolynomial3(torch.autograd.Function):

    def forward(ctx, input):
        ctx.save_for_backward(input)
        return 0.5 * (5 * input ** 3 - 3 * input)
    
    def backward(ctx,grad_output):
        input, = ctx.saved_tensors
        return grad_output * 1.5 * (5 * input ** 2 - 1) # chain law: `loss`相对于`predictions`（即网络输出）的梯度* `predictions`相对于`input`（即网络输入）的梯度 


In [ ]:


dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0")  # Uncomment this to run on GPU

# Create Tensors to hold input and outputs.

x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# Create random Tensors for weights.
a = torch.full((), 0.0, device=device, dtype=dtype, requires_grad=True)
b = torch.full((), -1.0, device=device, dtype=dtype, requires_grad=True)
c = torch.full((), 0.0, device=device, dtype=dtype, requires_grad=True)
d = torch.full((), 0.3, device=device, dtype=dtype, requires_grad=True)

learning_rate = 5e-6
for t in range(2000):
    P3 = LegendrePolynomial3.apply

    # Forward pass: compute predicted y 
    y_pred = a + b * P3(c + d * x)

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # Use autograd to compute the backward pass.
    loss.backward()

    # Update weights using gradient descent
    with torch.no_grad():
        a -= learning_rate * a.grad
        b -= learning_rate * b.grad
        c -= learning_rate * c.grad
        d -= learning_rate * d.grad

        # Manually zero the gradients after updating weights
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None

print(f'Result: y = {a.item()} + {b.item()} * P3({c.item()} + {d.item()} x)')

linear function

在PyTorch中，`unsqueeze`方法用于在指定的维度上为张量（tensor）增加一个维度，即在该维度上增加一个大小为1的新轴。这通常用于确保张量在某个维度上具有正确的大小，以便可以进行后续的运算。

例如，如果你有一个形状为`(n,)`的一维张量（即没有维度），并且你想要将其变成一个形状为`(1, n)`的二维张量，你可以在维度0上使用`unsqueeze`方法：

```python
vector = torch.tensor([1, 2, 3])  # 形状为 (3,)
matrix = vector.unsqueeze(0)      # 现在形状为 (1, 3)
```

`expand_as`方法的作用是将一个张量的形状“扩展”到另一个张量的形状，而不实际复制数据。当你使用`expand_as`时，PyTorch会检查两个张量的数据是否兼容，并且只有当它们可以不改变数据地广播到相同的形状时才会执行操作。
`output += bias.unsqueeze(0).expand_as(output)` 这个代码做了两个事情：

1. `bias.unsqueeze(0)`：在`bias`张量的维度0上增加一个新的轴，使其从形状`(D,)`变为`(1, D)`，其中`D`是`bias`的长度（即输出特征的数量）。

2. `.expand_as(output)`：将`bias`张量扩展到与`output`张量相同的形状。由于`bias`已经通过`unsqueeze`变成了二维张量，`expand_as`会将`bias`中的每个元素复制到`output`张量对应位置的整行中，从而使得`bias`能够作为`output`的加数。

@staticmethod
    def backward(ctx, grad_output):
        input, weight, bias = ctx.saved_tensors
        grad_input = grad_weight = grad_bias = None

        if ctx.needs_input_grad[0]:
            grad_input = grad_output.mm(weight)
        if ctx.needs_input_grad[1]:
            grad_weight = grad_output.t().mm(input)
        if bias is not None and ctx.needs_input_grad[2]:
            grad_bias = grad_output.sum(0).squeeze(0)

        return grad_input, grad_weight, grad_bias
        
它涉及到三个张量：input、weight 和 bias，这些张量可能是全连接层的输入特征、权重和偏置。在这个 backward 函数中，根据 ctx.needs_input_grad 属性，我们可以知道哪些输入张量需要计算梯度。然后，根据这些张量的梯度需求，函数计算并返回对应的梯度。


如果 input 需要梯度，grad_input 被计算为 grad_output 与 weight 的矩阵乘法结果。
如果 weight 需要梯度，grad_weight 被计算为 grad_output 的转置与 input 的矩阵乘法结果。
如果 bias 不是 None 并且需要梯度，grad_bias 被计算为 grad_output 在第一个维度上的求和，并压缩掉维度。

和这个的区别：
@staticmethod
def backward(ctx, grad_output):
    result, = ctx.saved_tensors
    return grad_output * result

在这个 backward 函数中，梯度的计算仅仅是将 grad_output 与 result 进行逐元素的乘法。这种类型的 backward 函数通常用于那些可以通过简单的逐元素操作来计算梯度的场景。
 

In [ ]:
import torch
import torch.nn as nn


class LinearFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input, weight, bias=None):
        ctx.save_for_backward(input, weight, bias)
        output = input.mm(weight.t())
        if bias is not None:
            output += bias.unsqueeze(0).expand_as(output)
        return output

    @staticmethod
    def backward(ctx, grad_output):
        input, weight, bias = ctx.saved_tensors
        grad_input = grad_weight = grad_bias = None

        if ctx.needs_input_grad[0]:
            grad_input = grad_output.mm(weight)
        if ctx.needs_input_grad[1]:
            grad_weight = grad_output.t().mm(input)
        if bias is not None and ctx.needs_input_grad[2]:
            grad_bias = grad_output.sum(0).squeeze(0)

        return grad_input, grad_weight, grad_bias


class Linear(nn.Module):
    def __init__(self, input_features, output_features, bias=True):
        super(Linear, self).__init__()
        self.input_features = input_features
        self.output_features = output_features

        self.weight = nn.Parameter(torch.Tensor(output_features, input_features))
        if bias:
            self.bias = nn.Parameter(torch.Tensor(output_features))
        else:
            self.register_parameter('bias', None)

        self.weight.data.uniform_(-0.1, 0.1)
        if bias is not None:
            self.bias.data.uniform_(-0.1, 0.1)

    def forward(self, input):
        # See the autograd section for explanation of what happens here.
        return LinearFunction.apply(input, self.weight, self.bias)


cross entropy

在你提供的代码中，定义了一个自定义的交叉熵损失函数（`CrossEntropyLoss`），并使用了PyTorch的自动微分机制。交叉熵损失是分类问题中常用的损失函数，特别是在多分类问题中。它衡量的是模型预测的概率分布与真实标签的概率分布之间的差异。

下面是交叉熵损失函数的一般公式：

设 \( \mathbf{y} \) 为真实标签的独热编码向量（one-hot vector），\( \mathbf{p} \) 为模型预测的概率分布（通过softmax函数得到的），那么对于每个样本 \( i \) 的交叉熵损失 \( L_i \) 可以表示为：

$ L_i(\mathbf{y}, \mathbf{p}) = -\sum_{j=1}^{M} y_{ij} \log(p_{ij}) $

其中 \( M \) 是类别的数量，\( y_{ij} \) 是独热编码向量 \( \mathbf{y} \) 中第 \( i \) 个样本的第 \( j \) 类的值（如果是第 \( j \) 类，则为1，否则为0），\( p_{ij} \) 是预测概率分布 \( \mathbf{p} \) 中第 \( i \) 个样本的第 \( j \) 类的概率。

对于整个批次的损失，我们通常取所有样本损失的平均值：

$ L = \frac{1}{N} \sum_{i=1}^{N} L_i(\mathbf{y}_i, \mathbf{p}_i) $

其中 \( N \) 是批次中样本的数量。

在代码中，`forward`方法首先将目标（`targets`）转换为独热编码向量，然后计算模型输出（`logits`）的softmax概率分布（\( \mathbf{p} \)），接着计算对数softmax（\( \log(\mathbf{p}) \)），并根据交叉熵损失的定义计算损失值。

在`backward`方法中，计算了相对于`logits`的梯度，这是通过以下公式得到的：

$ \frac{\partial L}{\partial \mathbf{z}} = \mathbf{p} - \mathbf{y} $

其中 \( \mathbf{z} \) 是`logits`的内部表示（在softmax函数内部使用），\( \mathbf{p} \) 是softmax概率分布，\( \mathbf{y} \) 是独热编码的目标向量。这个梯度告诉我们如何调整`logits`以减少损失。

在代码中，`grad_logits`是计算得到的梯度，它将用于更新`logits`（即模型的权重）。`grad_targets`在交叉熵损失的上下文中通常设置为`None`，因为目标的梯度对于模型训练没有影响（目标是固定的，不需要更新）。

最后，通过调用`auto_loss.backward()`，PyTorch会自动计算并存储`logits`相对于其梯度，这可以通过`logits.grad`访问。

In [ ]:
logits = torch.tensor([[0.2, 0.3, 0.9],
                  [0.2, 0.3, 0.9],], requires_grad=True)
targets = torch.tensor([0, 0])
class CrossEntropyLoss(torch.autograd.Function):
    @staticmethod
    def forward(ctx, logits, targets):
        targets = F.one_hot(targets, num_classes=logits.size(1)).float()
        prob = F.softmax(logits, 1)
        ctx.save_for_backward(prob, targets)
        logits = F.log_softmax(logits, 1)
        loss = -(targets * logits).sum(1).mean()
        return loss

    @staticmethod
    def backward(ctx, grad_output):
        prob, targets = ctx.saved_tensors
        grad_logits = (grad_output * (prob - targets)) / targets.size(0)
        grad_targets = None 
        return grad_logits, grad_targets
auto_loss =  CrossEntropyLoss.apply(x, target)
auto_loss.backward()
print(logits.grad)


exp

In [ ]:
class Exp(torch.autograd.Function):
    @staticmethod
    def forward(ctx, i):
        result = torch.exp(i)
        ctx.save_for_backward(result)
        return result

    @staticmethod
    def backward(ctx, grad_output):
        result, = ctx.saved_tensors
        return grad_output * result


# customize nn modules

自定义一个三阶多项式y = a+ bx + cx**2 +dx**3  来拟合y= sin(x)



In [ ]:
import torch
import math


class Polynomial3(torch.nn.Module):
    def __init__(self):
        """
        In the constructor we instantiate four parameters and assign them as
        member parameters.
        初始化四个参数
        """
        super().__init__()
        self.a = torch.nn.Parameter(torch.randn(()))
        self.b = torch.nn.Parameter(torch.randn(()))
        self.c = torch.nn.Parameter(torch.randn(()))
        self.d = torch.nn.Parameter(torch.randn(()))

    def forward(self, x):
        """
        计算前向传播
        """
        return self.a + self.b * x + self.c * x ** 2 + self.d * x ** 3

    def string(self):
        """
        Just like any class in Python, you can also define custom method on PyTorch modules
        """
        return f'y = {self.a.item()} + {self.b.item()} x + {self.c.item()} x^2 + {self.d.item()} x^3'


# Create Tensors to hold input and outputs.
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# Construct our model by instantiating the class defined above
model = Polynomial3()

# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters (defined
# with torch.nn.Parameter) which are members of the model.
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-6)
for t in range(2000):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(f'Result: {model.string()}')